In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pip install nltk

In [ ]:
data = pd.read_csv("data.csv") #file path

In [ ]:
data

In [ ]:
data.columns


In [ ]:
df = pd.DataFrame(data['Movie Name'])

In [ ]:
df['description'] = data['Description']

In [ ]:
df

In [ ]:
# The provided task is to process the text data and formulate recommendation function :

df.description

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Importing natural language toolkit functions to process text

In [ ]:
tokens = [word_tokenize(text) for text in df['description']]

In [ ]:
tokens

In [ ]:
# reducing the tokens into lower case letters :  (normalizing)


normalized_tokens = [[current.lower() for current in token] for token in tokens]

In [ ]:
# Now as the tokens are normalized, we can remove the english stop words from the text. 

stops = stopwords.words('english')

In [ ]:
# filtering the text normalized tokens to remove stop words on "stops"

In [ ]:
filtered_tokens = [[word for word in text] for text in tokens if text not in stops]

In [ ]:
filtered_tokens

In [ ]:
# the stop words are filtered out and filtered token list contains only non stop word lists.


# Punctuations should also be removed from the text. 

import string

In [ ]:
def remove_punctuation(words):
    return [word for word in words if word.isalpha() and word not in string.punctuation]

In [ ]:
filtered = [remove_punctuation(word) for word in filtered_tokens]

In [ ]:
filtered

In [ ]:
# Now the data is tokenized and vectorization can be done. 


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity   # for similarity index and recommendation

In [ ]:
df['filtered_description'] = filtered
df['filtered_description'] = df['filtered_description'].apply(lambda x: " ".join(x))

In [ ]:
df

In [ ]:
# vectorization

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df["filtered_description"])

In [ ]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
# now we need to prepare a recommendation function that will take the input of description string from the user
# recommend similar movies to the user. 



# 1. Take input -> pre-process and convert -> calculate similarity -> Recommend top 5.



In [ ]:
def recommend(input_string, df=df, vectorizer=vectorizer, tfidf_matrix=tfidf_matrix):

    # pre-processing the input text (same functions as done before)
    tokens = word_tokenize(input_string)  # tokenizing
    normalized_tokens = [current.lower() for current in tokens]  # Normalizing (to lower case)
    filtered_tokens = [word for word in normalized_tokens if word not in stops] # removing stop words
    filtered_input_tokens = remove_punctuation(filtered_tokens)
    filtered_string = " ".join(filtered_input_tokens)
    
    # vectorizing input 
    
    input_vector = vectorizer.transform([filtered_string])
    
    # calculating similarity between input and data vectors
    
    cosine_sim = cosine_similarity(input_vector, tfidf_matrix).flatten()
    
    # retreiving and returning top 5 movies
    
    sim_scores = sorted(list(enumerate(cosine_sim)), key=lambda x: x[1], reverse=True)
    top_movies = [df["Movie Name"].iloc[i[0]] for i in sim_scores[:5]]
    
    return top_movies
    

In [ ]:
# Example usage of the function and similarity model :: 


movie_description = df['description'][np.random.randint(0, len(df))]
print (movie_description)
print()
print("As per the description, the top 5 recommended picks are ") 
recommend(movie_description)